# Intro to Tensorflow

Below are the libraries needed for this course

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
import tensorflow as tf
from os import path, getcwd, chdir

# DO NOT CHANGE THE LINE BELOW. If you are developing in a local
# environment, then grab mnist.npz from the Coursera Jupyter Notebook
# and place it inside a local folder and edit the path to that location
path = f"{getcwd()}/../tmp2/mnist.npz"
config = tf.ConfigProto()

Below is a simple code to make a basic NN (Neural Network) to predict y value with the help of x value (feature)

In [ ]:
model = keras.Sequential({keras.layers.Dense(units=1,input_shape=[1])})
model.compile(optimizer = 'sgd', loss = 'mean_squared_error')

import numpy as np
xs = np.array([-1,0,1,2,3,4], dtype = 'float')
ys = np.array([-3,-1,1,3,5,7], dtype = 'float')
model.fit(xs,ys,epochs=500)

print(model.predict([10.0]))

## Computer vision using NN
Below is code for making a NN model to predict catergory of the particular image

Different types of activation which can be used
* relu - if x > 0 return x, else return 0
* softmax - To find max value - input([5,2,3,7,5,4,2]) => ouput([0,0,0,1,0,0,0])
* elu
* selu - scale x elu
* tanh 
* softplus - long(exp(x)+1)
* softsign - x/(abs(x)+1
* sigmoid - 1/(1+exp(-x))
* hardsigmoid - linear form of sigmoid
* linear
* exponential

Different types of Optimizers
* SGD 
    * learning_rate
    * momentum
    * nesterov 
* RMSprop
    * learing_rate
    * rho 
* Adagra
    * learning_rate
* Adam
    * learning_Rate
    * beta_1
    * beta_2
    * amsgrad
* Adamax
    * learning_Rate
    * beta_1
    * beta_2
* Nadam
    * learning_Rate
    * beta_1
    * beta_2
    
eg - 

sgd = keras.optimizer.SGD(learning_rate = 0.001, momnetum = 0.0, nesterov = False)
model.compute(optimizer = sgd, ......)


In [ ]:
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

training_images  = training_images / 255.0
test_images = test_images / 255.0

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('loss')<0.4):
            print("\nReached 60% accuracy so cancelling training!")
            self.model.stop_training = True
callbacks = myCallback()

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=5, callbacks=[callbacks])

model.evaluate(test_images, test_labels)

Below is code using convolutions creating CNN model

In [ ]:
import tensorflow as tf
print(tf.__version__)
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(training_images, training_labels, epochs=5)
test_loss = model.evaluate(test_images, test_labels)

Below is code using CNN 

In [ ]:
import os
import zipfile
from os import path, getcwd, chdir

path = f"{getcwd()}/../tmp2/happy-or-sad.zip"

zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

def train_happy_sad_model():

    DESIRED_ACCURACY = 0.999

    class myCallback(tf.keras.callbacks.Callback):
         def on_epoch_end(self,epch,logs={}):
                if(logs.get('acc')>=DESIRED_ACCURACY):
                    self.model.stop_training = True

    callbacks = myCallback()
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    from tensorflow.keras.optimizers import RMSprop
    model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])
        


    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale = 1./255)# Your Code Here

    train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s',  
        target_size=(300, 300),  
        batch_size=128,
        class_mode='binary')
    
    history = model.fit_generator(train_generator,steps_per_epoch=5,epochs=15,verbose=1,callbacks=[callbacks])

    return history.history['acc'][-1]